디자이너 에이전트가 쿼리에 맞게 디자인 계획을 세우고 계획을 달성하는 과정에서 ColorGen이라는 도구를 사용합니다. 

In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
from langgraph.checkpoint.memory import MemorySaver

# 메모리 저장소 생성
memory = MemorySaver()

In [2]:
from langgraph.graph.message import add_messages
from typing import Annotated
from typing_extensions import TypedDict

class State(TypedDict):
    messages: Annotated[list, add_messages]

In [3]:
from typing import Annotated
from typing_extensions import TypedDict
from langchain_openai import ChatOpenAI
from langchain_teddynote.tools.tavily import TavilySearch
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

In [4]:
from langchain_core.tools import tool
from typing import List, Dict

# 도구 생성
@tool
def color_gen(query: str) -> List[Dict[str, str]]:
    """generate color based on the query"""

    if query == "warm":
        return [{"red": "#FF0000"}, {"orange": "#FFA500"}, {"yellow": "#FFFF00"}]

    return [{"blue": "#0000FF"}, {"green": "#008000"}, {"purple": "#800080"}]

@tool
def font_gen(query: str) -> List[Dict[str, str]]:
    """generate font based on the query"""

    if query == "warm":
        return [{"Arial": "sans-serif"}, {"Times New Roman": "serif"}, {"Courier New": "monospace"}]

    return [{"Helvetica": "sans-serif"}, {"Georgia": "serif"}, {"Comic Sans MS": "cursive"}]

In [5]:
from langgraph.prebuilt import ToolNode, tools_condition

# 도구 리스트 생성
tools = [color_gen, font_gen]

# ToolNode 초기화
tool_node = ToolNode(tools)

In [6]:
from langchain_core.messages import AIMessage

# 단일 도구 호출을 포함하는 AI 메시지 객체 생성
# AIMessage 객체이어야 함
message_with_single_tool_call = AIMessage(
    content="",
    tool_calls=[
        {
            "name": "color_gen",  # 도구 이름
            "args": {"query": "warm"},  # 도구 인자
            "id": "tool_call_id",  # 도구 호출 ID
            "type": "tool_call",  # 도구 호출 유형
        }
    ],
)

# 도구 노드를 통한 메시지 처리 및 날씨 정보 요청 실행
tool_node.invoke({"messages": [message_with_single_tool_call]})

{'messages': [ToolMessage(content='[{"red": "#FF0000"}, {"orange": "#FFA500"}, {"yellow": "#FFFF00"}]', name='color_gen', tool_call_id='tool_call_id')]}

In [7]:
from langchain_openai import ChatOpenAI

# LLM 모델 초기화 및 도구 바인딩
model_with_tools = ChatOpenAI(model="gpt-4o-mini", temperature=0).bind_tools(tools)

In [ ]:
# model_with_tools.invoke(state["messages"])

In [55]:
# 도구 호출 확인
model_with_tools.invoke("따뜻한 그림에는 어떤 색이 어울릴까?").tool_calls

[{'name': 'color_gen',
  'args': {'query': '따뜻한 그림'},
  'id': 'call_HB5oXCU6JmN6SrGWtqlLjWeA',
  'type': 'tool_call'}]

In [8]:
# 도구 노드를 통한 메시지 처리 및 LLM 모델의 도구 기반 응답 생성
tool_node.invoke(
    {
        "messages": [
            model_with_tools.invoke(
                "따뜻한 그림에는 어떤 색이 어울릴까?"
            )
        ]
    }
)

{'messages': [ToolMessage(content='[{"blue": "#0000FF"}, {"green": "#008000"}, {"purple": "#800080"}]', name='color_gen', tool_call_id='call_pHhSWZ8piDGSsUAQmZTa3dh5')]}

In [10]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage

def call_designer(messages: List[BaseMessage]) -> dict:
    # LangChain ChatOpenAI 모델을 Agent 로 변경할 수 있습니다.
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a professional designer whose expertise lies in creating a wide range of promotional materials—everything from brochures and flyers to social media graphics, email headers, posters, and banners. Your task is to develop eye-catching, on-brand assets that effectively communicate the client’s message and drive engagement.",
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )
    chain = prompt | model_with_tools # | StrOutputParser()
    return chain.invoke({"messages": messages}) 

# call_designer([("user", "안녕하세요? 당신은 누구입니까?")])
call_designer([("user", "안녕하세요? 따뜻한 톤의 그림을 위해서는 어떠한 색이 어울릴까요?")])

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_4CRUqq2rAe0qaUXNNcjYEfQL', 'function': {'arguments': '{"query":"따뜻한 톤의 색상"}', 'name': 'color_gen'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 145, 'total_tokens': 167, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_129a36352a', 'id': 'chatcmpl-BVyIs5yLhrxkpWjCKjKsdzG78xIge', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-bd003940-8683-4908-ae0f-b32966bf231e-0', tool_calls=[{'name': 'color_gen', 'args': {'query': '따뜻한 톤의 색상'}, 'id': 'call_4CRUqq2rAe0qaUXNNcjYEfQL', 'type': 'tool_call'}], usage_metadata={'input_tokens': 145, 'output_tokens': 22, 'total_tokens': 167, 'input_token_details': {'audio': 0, 'cache_read':

In [65]:
# LangGraph 워크플로우 상태 및 메시지 처리를 위한 타입 임포트
from langgraph.graph import StateGraph, START, END

# 상담사 역할
def designer_node(messages: State) -> State:
    # 상담사 응답 호출
    ai_response = call_designer(messages["messages"])
    print(type(ai_response))

    # AI 상담사의 응답을 반환
    return {"messages": [AIMessage(ai_response)]}

# 메시지 상태 기반 워크플로우 그래프 초기화
workflow = StateGraph(State)

# 에이전트와 도구 노드 정의 및 워크플로우 그래프에 추가
workflow.add_node("designer", designer_node)
workflow.add_node("tools", tool_node)

# 워크플로우 시작점에서 에이전트 노드로 연결
workflow.add_edge(START, "designer")

# 에이전트 노드에서 조건부 분기 설정, 도구 노드 또는 종료 지점으로 연결
workflow.add_conditional_edges("designer", tools_condition)

# 도구 노드에서 에이전트 노드로 순환 연결
workflow.add_edge("tools", "designer")

# 에이전트 노드에서 종료 지점으로 연결
workflow.add_edge("designer", END)


# 정의된 워크플로우 그래프 컴파일 및 실행 가능한 애플리케이션 생성
app = workflow.compile()

In [66]:
from langchain_core.runnables import RunnableConfig
from langchain_teddynote.messages import stream_graph, random_uuid


# config 설정(재귀 최대 횟수, thread_id)
config = RunnableConfig(recursion_limit=10, configurable={"thread_id": random_uuid()})

# 입력 메시지 설정
inputs = {
    "messages": [HumanMessage(content="안녕하세요? 따뜻한 톤의 그림을 위해서는 어떠한 색이 어울릴까요?")]
}

# 그래프 스트리밍
stream_graph(app, inputs, config)


🔄 Node: designer 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
<class 'langchain_core.messages.ai.AIMessage'>


ValidationError: 21 validation errors for AIMessage
content.str
  Input should be a valid string [type=string_type, input_value=AIMessage(content='', add..., 'type': 'tool_call'}]), input_type=AIMessage]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type
content.list[union[str,dict[any,any]]].0.str
  Input should be a valid string [type=string_type, input_value=('content', ''), input_type=tuple]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type
content.list[union[str,dict[any,any]]].0.dict[any,any]
  Input should be a valid dictionary [type=dict_type, input_value=('content', ''), input_type=tuple]
    For further information visit https://errors.pydantic.dev/2.11/v/dict_type
content.list[union[str,dict[any,any]]].1.str
  Input should be a valid string [type=string_type, input_value=('additional_kwargs', {'t..., 'type': 'function'}]}), input_type=tuple]
    For further information visit https://errors.pydantic.dev/2.11/v/strin